# Money Aggregates Vocabulary (MoneyAggVoc)

## Transform CSV into RDF

In [1]:
import re
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, SKOS, DCTERMS, NamespaceManager

In [2]:
df = pd.read_csv('moneyaggvoc.csv', index_col = '@id', keep_default_na = False)
df

,rdf:type,skos:hasTopConcept,skos:inScheme,skos:broader,skos:related,skos:exactMatch,rdfs:label,rdfs:comment,dcterms:creator,dcterms:source,skos:prefLabel,skos:altLabel,skos:definition,skos:note
@id,,,,,,,,,,,,,,
mav:scheme,skos:ConceptScheme,mav:mav001,,,,,Money Aggregate Vocabulary,"Money Aggregate Vocabulary, or MoneyAggVoc for...",https://www.linkedin.com/in/emorozov/,,,,,
mav:mav001,skos:Concept,,mav:scheme,,,,,,,http://dbpedia.org/resource/Money_supply,Money aggregate,,"In economics, the money supply or money stock,...",
mav:mav002,skos:Concept,,mav:scheme,mav:mav001,,http://eurovoc.europa.eu/1825,,,,https://www.ecb.europa.eu/stats/money_credit_b...,Eurozone money aggregate,,The ECB's definition of euro area monetary agg...,
mav:mav003,skos:Concept,,mav:scheme,mav:mav002,,,,,,https://www.ecb.europa.eu/stats/money_credit_b...,M1,Narrow money,"Narrow money (M1) includes currency, i.e. bank...",
mav:mav004,skos:Concept,,mav:scheme,mav:mav002,,,,,,https://www.ecb.europa.eu/stats/money_credit_b...,M2,Intermediate money,Intermediate money (M2) comprises narrow money...,
mav:mav005,skos:Concept,,mav:scheme,mav:mav002,,,,,,https://www.ecb.europa.eu/stats/money_credit_b...,M3,Broad money,Broad money (M3) comprises M2 and marketable i...,
mav:mav006,skos:Concept,,mav:scheme,mav:mav001,,,,,,https://www.federalreserve.gov/faqs/money_1284...,Unites States money aggregate,,The money supply is commonly defined to be a g...,
mav:mav007,skos:Concept,,mav:scheme,mav:mav006,,,,,,https://www.federalreserve.gov/faqs/money_1284...,Monetary base,,The monetary base is defined as the sum of cur...,This measure is not listed in Economics CFA Pr...
mav:mav008,skos:Concept,,mav:scheme,mav:mav006,,,,,,https://www.federalreserve.gov/faqs/money_1284...,M1,,M1 is defined as the sum of currency held by t...,


In [3]:
# Graph to store the set of schemas
moneyaggvoc_graph = Graph()

# Declare all namespaces
MAV = Namespace('http://cadmiumkitty.github.io/moneyaggvoc/')

# Bind namespaces for clean serialization
moneyaggvoc_namespace_manager = NamespaceManager(moneyaggvoc_graph)
moneyaggvoc_namespace_manager.bind('rdfs', RDFS, override = False)
moneyaggvoc_namespace_manager.bind('skos', SKOS, override = False)
moneyaggvoc_namespace_manager.bind('dcterms', DCTERMS, override = False)
moneyaggvoc_namespace_manager.bind('mav', MAV, override = False)

In [4]:
# RdfLib gives back prefixes as a generator, which is inconvenient to work with when parsing the 
prefixes = {}
for prefix, uri in moneyaggvoc_namespace_manager.namespaces():
    prefixes[prefix] = uri
prefixes

{'xml': rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'),
 'rdf': rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
 'rdfs': rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'),
 'xsd': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'),
 'skos': rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#'),
 'dcterms': rdflib.term.URIRef('http://purl.org/dc/terms/'),
 'mav': rdflib.term.URIRef('http://cadmiumkitty.github.io/moneyaggvoc/')}

In [5]:
# Simple definitions of the curie and uri since we know what is in the dataset
curie = re.compile("^\w*:\w*$")
uri = re.compile('^http[s]?://.*$')

In [6]:
# Define simple function to transform Pandas value into RdfLib Node (eitehr Literal or URIRef)
# taking declared prefixes into account
# We assume that all literals are strings for now
def getNode(value):
    if (curie.match(value)):
        prefix, reference = value.split(':')        
        uriref = ''.join((prefixes[prefix], reference))
        return URIRef(uriref)
    elif uri.match(value):
        return URIRef(value)
    else:
        return Literal(value)

In [7]:
for (index, series) in df.iterrows():
    for (column, value) in series.iteritems():
        if value:
            moneyaggvoc_graph.add((getNode(index),
                   getNode(column),
                   getNode(value)))

In [8]:
# Serialize MoneyAggVoc into ttl
moneyaggvoc_ttl = moneyaggvoc_graph.serialize(format = 'turtle', indent = 2)
moneyaggvoc_ttl

b'@prefix dcterms: <http://purl.org/dc/terms/> .\n@prefix mav: <http://cadmiumkitty.github.io/moneyaggvoc/> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix skos: <http://www.w3.org/2004/02/skos/core#> .\n@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nmav:mav003 a skos:Concept ;\n    dcterms:source <https://www.ecb.europa.eu/stats/money_credit_banking/monetary_aggregates/html/hist_content.en.html> ;\n    skos:altLabel "Narrow money" ;\n    skos:broader mav:mav002 ;\n    skos:definition "Narrow money (M1) includes currency, i.e. banknotes and coins, as well as balances which can immediately be converted into currency or used for cashless payments, i.e. overnight deposits." ;\n    skos:inScheme mav:scheme ;\n    skos:prefLabel "M1" .\n\nmav:mav004 a skos:Concept ;\n    dcterms:source <https://www.ecb.europa.eu/stats/money_credit_banking/monetary_a

In [9]:
# Write out the ttl into a file
moneyaggvoc_file_name = 'moneyaggvoc.ttl'
with open(moneyaggvoc_file_name,'wb') as moneyaggvoc_file:
    moneyaggvoc_file.write(moneyaggvoc_ttl)